In [1]:
pip install pandas matplotlib scikit-learn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv('combined-divvy-tripdata-with-weather.csv')
df2 = pd.read_csv('weather.csv')

df['datetime'] = pd.to_datetime(df['started_at'])
df['date'] = df['datetime'].dt.date
df2['date'] = pd.to_datetime(df2['started_at'])
df2['date'] = df2['date'].dt.date
df3 = df2[['date', 'weather']]
df = pd.merge(df, df3, on='date', how='left')
print(df)

: 

In [ ]:
df.drop(columns=['datetime','date'], inplace=True)

In [ ]:
df['TAVG'] = (df['TMIN'] + df['TMAX']) / 2


In [ ]:
df.to_csv('/content/combined-divvy-tripdata-with-weather.csv', index=False)

In [ ]:
import shutil
shutil.copy('/content/combined-divvy-tripdata-with-weather.csv', '/content/drive/MyDrive/517')

'/content/drive/MyDrive/517/combined-divvy-tripdata-with-weather.csv'

In [ ]:
import matplotlib
matplotlib.use('Agg')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels
from statsmodels.genmod.families import Poisson
from statsmodels.tools.eval_measures import rmse

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

import pandas as pd

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/517/combined-divvy-tripdata-with-weather.csv')

df['started_at'] = pd.to_datetime(df['started_at'])

# Function to categorize season
def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

# Function to categorize part of the day
def get_part_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df.dropna(inplace=True)
# Apply functions to categorize 'started_at' into new columns
df['day_of_week'] = df['started_at'].dt.day_name()
df['season'] = df['started_at'].dt.month.apply(get_season)
df['part_of_day'] = df['started_at'].dt.hour.apply(get_part_of_day)
df['same_start_end'] = (df['start_station_name'] == df['end_station_name'])


# Aggregate data if necessary, summing up the number of trips for each member type
# If your data is already aggregated, you might not need this step
df = df.groupby(['member_casual', 'day_of_week', 'season', 'part_of_day', 'same_start_end', 'rideable_type']).agg(
    TMIN_avg=('TMIN', 'mean'),
    TMAX_avg=('TMAX', 'mean'),
    num_trips = ('TMIN', 'size')
).reset_index()
print(df.head(5))

df = pd.concat((df, pd.get_dummies(df['member_casual'])), axis=1)
df = pd.concat((df, pd.get_dummies(df['day_of_week'])), axis=1)
df = pd.concat((df, pd.get_dummies(df['season'])), axis=1)
df = pd.concat((df, pd.get_dummies(df['part_of_day'])), axis=1)
df = pd.concat((df, pd.get_dummies(df['same_start_end'])), axis=1)
# df = pd.concat((df, pd.get_dummies(df['TMIN_avg'])), axis=1)
# df = pd.concat((df, pd.get_dummies(df['TMAX_avg'])), axis=1)
df = pd.concat((df, pd.get_dummies(df['rideable_type'])), axis=1)
x1 = pd.DataFrame()
x1 = df
x1.drop(columns=['member_casual','day_of_week', 'season', 'part_of_day', 'same_start_end', 'rideable_type', 'TMIN_avg', 'TMAX_avg'], inplace=True)
y = df['num_trips']
x = df[x1.columns]

x = sm.add_constant(x)
pm = sm.GLM(y, x, family=sm.families.Poisson()).fit()
print("poisson regression's rmse value")
print(sm.tools.eval_measures.rmse(y, pm.fittedvalues, axis=0))

######################################################################

from sklearn.metrics import mean_squared_error

# Define your train and test sets
size = 0.2
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=size, random_state=42)

# Fit the Poisson regression model on the training data
pm_train = sm.GLM(y_train, x_train, family=sm.families.Poisson()).fit()

# Predict on training data and calculate RMSE
train_preds = pm_train.predict(x_train)
train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
print(f"Train RMSE: {train_rmse}")

# Predict on test data and calculate RMSE
test_preds = pm_train.predict(x_test)  # Use the model trained on training data to predict test data
test_rmse = np.sqrt(mean_squared_error(y_test, test_preds))
print(f"Test RMSE: {test_rmse}")

# Plotting actual vs predicted values for the test set
plt.figure(figsize=(10, 6))  # Increase figure size for better visibility
plt.plot(y_test.values[:100], 'b-', label='Actual', linewidth=2)  # Plot the first 100 actual values
plt.plot(test_preds.values[:100], 'r--', label='Predicted', linewidth=2)  # Plot the first 100 predicted values
plt.legend()
plt.title("Actual vs Predicted")
plt.xlabel("Sample Index")
plt.ylabel("Number of Trips")
plt.show()

# Save the plot
plt.savefig("line_comparison.png")
plt.clf()


  member_casual day_of_week  season part_of_day  same_start_end  \
0        casual      Friday  Autumn   Afternoon           False   
1        casual      Friday  Autumn   Afternoon           False   
2        casual      Friday  Autumn   Afternoon           False   
3        casual      Friday  Autumn   Afternoon            True   
4        casual      Friday  Autumn   Afternoon            True   

   rideable_type   TMIN_avg   TMAX_avg  num_trips  
0   classic_bike  53.898828  73.405182      10151  
1    docked_bike  52.481124  72.832634       2384  
2  electric_bike  53.019428  72.574501      11530  
3   classic_bike  53.709427  73.784257       1029  
4    docked_bike  55.311953  75.483965        343  
poisson regression's rmse value
1417.3440193125393
Train RMSE: 1446.9200886141791
Test RMSE: 1285.9701726353092
